# Creating an Airport Based Feature Vector

In this notebook, we will build upon the understanding gathered during the data analysis and create a feature vector that is supposed to be fed into a learning algorithm.

In [30]:
import os.path
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import pandas as pd
import json
import requests
from IPython.display import clear_output
import cesiumpy
import random
from geomet import wkt
from pandas.io.json import json_normalize, read_json
from SPARQLWrapper import SPARQLWrapper, JSON, XML, RDF
from datetime import datetime
from IPython.display import HTML

#Set some parameters for nicer visualizations
pd.set_option('display.expand_frame_repr', False) #do not wrap the printout of Pandas DataFrames
pd.set_option('display.precision', 2)
mp.rcParams['figure.figsize'] = (15, 9)
mp.pyplot.style.use = False
%matplotlib inline

# initialize my connection module which allows to connect oto both datAcron graph databases
from datacron_connector import TripleStoreConnector
ts107 = TripleStoreConnector(0)
ts109 = TripleStoreConnector(1)

In [2]:
level3airports = 'data/level3airports.csv'
if not os.path.isfile(level3airports):
    qry = """
    PREFIX : <http://www.datacron-project.eu/datAcron#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
    PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

    SELECT ?s ?icao ?iata ?wkt 
    WHERE{
        VALUES ?icao { 'LEMD' 'LEBL' 'LEPA' 'LEBB' 'LEAL' 'LEMG' 'LEVC'}
        ?s :hasICAOcode ?icao;
           :hasIATAcode ?iata;
           :hasGeometry/:hasMBR_WKT ?wkt.
    }
    """
    dfad = ts107.query(qry)
    dfad = ts107.clean(dfad)
    dfad['geojson'] = dfad['wkt'].apply(lambda x: wkt.loads(x))
    dfad['sta'] = '2016-04-01T00:00'
    dfad['end'] = '2016-04-30T23:59:59'
    dfad['sta'] = pd.to_datetime(dfad['sta'])
    dfad['end'] = pd.to_datetime(dfad['end'])
    dfad['duration'] = dfad['end'] - dfad['sta']
    dfad.to_csv(level3airports)
else:
    dfad = pd.read_csv(level3airports, index_col=0)
    import ast
    dfad['geojson'] = dfad['geojson'].map(ast.literal_eval) #convert string to dict
    dfad['sta'] = pd.to_datetime(dfad['sta'])
    dfad['end'] = pd.to_datetime(dfad['end'])
    dfad['duration'] = pd.to_timedelta(dfad['duration'])
    
dfad.set_index('icao', inplace=True)   

In [3]:
#rwys = pd.Series([280, 250, 240, 180, 320, 190, 240, 300], index = [0, 1, 2, 3, 4, 5, 6])
rwys = pd.Series({'LEAL': 280, 'LEBB': 300, 'LEBL': 250, 'LEMD': 180, 'LEMG': 320, 'LEPA': 240, 'LEVC': 300 })
rwys = pd.DataFrame(rwys)
rwys = rwys.rename(columns={0: 'rwys'})
dfad = pd.merge(dfad, rwys, left_index=True, right_index=True)

## Slicing into 1 hour timeframes

Now we have the basic airport listings for the major spanish level 3 airports and two additionaly columns representing a validity for the AIRAC cycles 411 and 412. As in the notebook 5.1, the following procedure will generate a (huge) DataFrame, derivec from `dfad` by slicing the rows into mulitple rows, each containing the airport reference, but a validity of max. 1 hour.

In [4]:
mytimedelta = pd.Timedelta('1 hour')

#Create a boolean mask with rows to split (a vector of (True, True, False, ...) of length len(df))
split_rows = (dfad['duration'] > mytimedelta)    
i = 1

while split_rows.any():
    #get new rows to append and adjust start time to 1 hour later.
    new_rows = dfad[split_rows].copy()
    new_rows['sta'] = new_rows['sta'] + mytimedelta

    #update the end time of old rows
    dfad.loc[split_rows, 'end'] = dfad.loc[split_rows, 'sta'] + \
        pd.DateOffset(hours=1, seconds=-1)
    dfad = dfad.append(new_rows)
    
    #update the duration of all rows
    dfad['duration'] = dfad['end'] - dfad['sta']
    
    #create an updated boolean mask
    split_rows = (dfad['duration'] > mytimedelta)
    
    #give info to user
    clear_output(wait=True)
    i = i + 1
    print('Iteration' + str(i) + ' done.')

# reset index and make icao code a normal column again
dfad = dfad.reset_index(drop=False)
dfad = dfad.rename(columns={'index': 'icao'})
    

Iteration720 done.


# Enriching the dataset with weather information

As we now have the airport and some time sliced data, we can now enrich the dataset by inserting weather, capacity and demand information. We will start off with the weather information, taken from the `darksky.net` API. 

# TODO

at the moment, this is the point where we reduce the dataset to smaller complexity with head(200)





In [6]:
dfwx = dfad.head(100).copy()

dfwx['wx'] = ""
for index, row in dfwx.iterrows():
    airport = row['s']
    tim = row['sta'].strftime('%Y-%m-%dT%H:%M:%S') +'Z'
    lon = format(row['geojson']['coordinates'][0], '.2f')
    lat = format(row['geojson']['coordinates'][1], '.2f')
    qry = "https://api.darksky.net/forecast/5456d39a8263a71b4a425f871205fc67/"
    qry = qry + "{la},{lo},{ti}?exclude=hourly,daily&units=si".format(la = str(lat), lo = str(lon), ti = tim)
    ret = requests.get(qry)
    clear_output(wait=True)
    print('iteration ' + str(index) + ' done: response 200 is good:' )
    print(ret)
    dfwx.at[index, 'wx'] = ret.json()


iteration 99 done: response 200 is good:
<Response [200]>


Now we are going to place the received wx JSON data into individual columns for our feature vector. First, we create the columns with standard values according to the ISA standard atmosphere: https://en.wikipedia.org/wiki/International_Standard_Atmosphere. Second, we extract the wx column into a temporary dataFrame of the same length, and overwrite the ISA standard information with the information given in that DataFrame. Values not available in the JSAON are being taken care of by the `json_normalize` function and converted to NaN.

In [7]:
dftemp = pd.io.json.json_normalize(dfwx['wx'])

dfwx['humidity'] = 0.0
dfwx['dewPoint'] = -15.0
dfwx['temperature'] = 15.0
dfwx['pressure'] = 1013.25
dfwx['visibility'] = 10.0
dfwx['windBearing'] = 0
dfwx['windSpeed'] = 0.0
dfwx['windGust'] = 0.0
dfwx['precipIntensity'] = 0.0
dfwx['precipProbability'] = 0.0
dfwx['storm'] = 0
dfwx['humidity'] = dftemp['currently.humidity']
dfwx['dewPoint'] = dftemp['currently.dewPoint']
dfwx['temperature'] = dftemp['currently.temperature']
dfwx['pressure'] = dftemp['currently.pressure']
dfwx['visibility'] = dftemp['currently.visibility']
dfwx['windBearing'] = dftemp['currently.windBearing']
dfwx['windSpeed'] = dftemp['currently.windSpeed']
dfwx['precipIntensity'] = dftemp['currently.precipIntensity']
dfwx['precipProbability'] = dftemp['currently.precipProbability']

#dfwx['storm'] = dftemp['currently.nearestStormDistance']
#dfwx['windGust'] = dftemp['currently.windGust']

,icao,s,iata,wkt,geojson,sta,end,duration,rwys,wx,...,dewPoint,temperature,pressure,visibility,windBearing,windSpeed,windGust,precipIntensity,precipProbability,storm
0,LEAL,Place_Alicante_Elche_Airport,ALC,POINT (-0.558055579662323 38.282222747802734),"{'type': 'Point', 'coordinates': [-0.558055579...",2016-04-01,2016-04-01 00:59:59,00:59:59,280,"{'latitude': 38.28, 'longitude': -0.56, 'timez...",...,-2.58,15.93,1014.35,15.13,310,4.03,0.0,0.0,0.0,0
1,LEBL,Place_Barcelona_ElPrat_Airport,BCN,POINT (2.0783333778381348 41.29694366455078),"{'type': 'Point', 'coordinates': [2.0783333778...",2016-04-01,2016-04-01 00:59:59,00:59:59,250,"{'latitude': 41.3, 'longitude': 2.08, 'timezon...",...,9.24,12.49,1012.15,10.32,39,4.28,0.0,0.0,0.0,0
2,LEBB,Place_Bilbao___Sondica,BIO,POINT (-2.910555601119995 43.301109313964844),"{'type': 'Point', 'coordinates': [-2.910555601...",2016-04-01,2016-04-01 00:59:59,00:59:59,300,"{'latitude': 43.3, 'longitude': -2.91, 'timezo...",...,2.89,8.21,1022.95,12.71,330,3.58,0.0,0.0,0.0,0
3,LEMD,Place_Madrid_Barajas_Airport,MAD,POINT (-3.560833215713501 40.47222137451172),"{'type': 'Point', 'coordinates': [-3.560833215...",2016-04-01,2016-04-01 00:59:59,00:59:59,180,"{'latitude': 40.47, 'longitude': -3.56, 'timez...",...,-2.21,4.67,1021.25,16.08,351,5.04,0.0,0.0,0.0,0


An important step in preparing the wx data is to determine the crosswind component. The crosswind component is the sine of the wind angle times the wind speed. Consequently, angles greater than 90° will lead to smaller crosswind components. Tail- or Headwind components are disregarded here, as in most cases, ATC will then decide to switch the runway direction. Therefore, the CWC for our purpose is calculated as follows. Let $r$ be the runway direction and $wb$ be the wind bearing, both in degrees from 0° to 360°. Let $ws$ be the wind speed.

 $$ CWC = \lvert \sin( (r - wb) \cdot \frac{\pi}{180} ) \rvert \cdot ws $$

First, we want to create another column with the main runway direction from which we can infer crosswind components later on.

In [8]:
import math

def fcrosswind(rwy, windBearing, windSpeed):
    return math.fabs(math.sin((rwy - windBearing) * math.pi / 180)) * windSpeed

dfwx['crosswind'] = dfwx[['rwys', 'windBearing', 'windSpeed']]. apply(lambda x: fcrosswind(*x), axis = 1)

,icao,s,iata,wkt,geojson,sta,end,duration,rwys,wx,...,temperature,pressure,visibility,windBearing,windSpeed,windGust,precipIntensity,precipProbability,storm,crosswind
0,LEAL,Place_Alicante_Elche_Airport,ALC,POINT (-0.558055579662323 38.282222747802734),"{'type': 'Point', 'coordinates': [-0.558055579...",2016-04-01,2016-04-01 00:59:59,00:59:59,280,"{'latitude': 38.28, 'longitude': -0.56, 'timez...",...,15.93,1014.35,15.13,310,4.03,0.0,0.0,0.0,0,2.01
1,LEBL,Place_Barcelona_ElPrat_Airport,BCN,POINT (2.0783333778381348 41.29694366455078),"{'type': 'Point', 'coordinates': [2.0783333778...",2016-04-01,2016-04-01 00:59:59,00:59:59,250,"{'latitude': 41.3, 'longitude': 2.08, 'timezon...",...,12.49,1012.15,10.32,39,4.28,0.0,0.0,0.0,0,2.20
2,LEBB,Place_Bilbao___Sondica,BIO,POINT (-2.910555601119995 43.301109313964844),"{'type': 'Point', 'coordinates': [-2.910555601...",2016-04-01,2016-04-01 00:59:59,00:59:59,300,"{'latitude': 43.3, 'longitude': -2.91, 'timezo...",...,8.21,1022.95,12.71,330,3.58,0.0,0.0,0.0,0,1.79
3,LEMD,Place_Madrid_Barajas_Airport,MAD,POINT (-3.560833215713501 40.47222137451172),"{'type': 'Point', 'coordinates': [-3.560833215...",2016-04-01,2016-04-01 00:59:59,00:59:59,180,"{'latitude': 40.47, 'longitude': -3.56, 'timez...",...,4.67,1021.25,16.08,351,5.04,0.0,0.0,0.0,0,0.79
4,LEMG,Place_Malaga_CostaDelSol_Airport,AGP,POINT (-4.499166488647461 36.67499923706055),"{'type': 'Point', 'coordinates': [-4.499166488...",2016-04-01,2016-04-01 00:59:59,00:59:59,320,"{'latitude': 36.67, 'longitude': -4.5, 'timezo...",...,12.48,1017.89,15.98,290,6.55,0.0,0.0,0.0,0,3.27


Finally, we want to fill the NaN values with values that make sense. The darksky API will return some values only if they were applicable. As some weather events like "storms" are extremely rare, dropping the rows without those conditions would mean to drop an unacceptable amount of data. Therefore, we decide to substitute the NaN values with the column mean.

In [10]:
dfwx.fillna(dfwx.mean(), inplace=True)
dfwx.describe()

,duration,rwys,humidity,dewPoint,temperature,pressure,visibility,windBearing,windSpeed,windGust,precipIntensity,precipProbability,storm,crosswind
count,100,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.0,100.00,100.00,100.0,100.00
mean,0 days 00:59:59,267.10,0.60,2.51,11.16,1016.14,11.69,220.29,3.92,0.0,0.03,0.01,0.0,1.65
std,0 days 00:00:00,44.14,0.22,4.67,4.06,3.49,3.01,118.43,2.09,0.0,0.21,0.05,0.0,1.66
min,0 days 00:59:59,180.00,0.20,-3.84,1.71,1011.71,5.41,0.00,0.03,0.0,0.00,0.00,0.0,0.00
25%,0 days 00:59:59,240.00,0.40,-2.13,8.89,1013.75,9.99,106.25,2.31,0.0,0.00,0.00,0.0,0.27
50%,0 days 00:59:59,280.00,0.59,2.26,11.83,1014.57,10.54,280.50,4.02,0.0,0.00,0.00,0.0,1.15
75%,0 days 00:59:59,300.00,0.81,7.40,12.78,1018.31,15.05,310.25,5.48,0.0,0.00,0.00,0.0,2.65
max,0 days 00:59:59,320.00,0.94,10.01,20.63,1022.96,16.09,359.00,8.50,0.0,1.92,0.38,0.0,7.46


## Capacity computation

As we have seen in the prior work, demand and capacity have to be considered for the wx regulation detection problem. The following lines of code will enrich the dataset with a demand/capacity ratio for each row. The capacity values are taken from the AECFA document provided by CRIDA.

Note to self:
Capacity definitions that cross midnight have to be divided into a pre-midnight and a post-midnight-entry, otherwise the comparison function in the double for-Loop might fail sometimes.

Note 2 to self:
In this notebook, we can guarantee that all rowes have the exact duration of one hour. Therefore, no factorization of the capacity values - as in notebooks 5.1 - is needed.

In [11]:
dfcap = pd.read_csv('data/airport_capacities_aecfa.csv')
dfcap['start'] = pd.to_datetime(dfcap['start'])
dfcap['end'] = pd.to_datetime(dfcap['end'])
dfcap.sample()

,icao,start,end,cap
3,LEBL,2017-11-19 22:00:00,2017-11-19 23:59:59,48


Note that we do not care that the above oepration returned the current date, as we are interested only in the hours of the day, which we will access via the `.time()` property of the datetime object.
Now we can iterate through the dfwx dataset and insert the proper capacity values from dfcap. 

In [14]:
dfwx['cap'] = 999

for index, wxrow in dfwx.iterrows():
    for index2, caprow in dfcap.iterrows():
        if wxrow['icao'] == caprow['icao']:
            if ((wxrow['sta'].time() >= caprow['start'].time()) & (wxrow['sta'].time() <= caprow['end'].time())):
                dfwx.at[index, 'cap'] = caprow['cap']

dfwx.head(4)

,icao,s,iata,wkt,geojson,sta,end,duration,rwys,wx,...,pressure,visibility,windBearing,windSpeed,windGust,precipIntensity,precipProbability,storm,crosswind,cap
0,LEAL,Place_Alicante_Elche_Airport,ALC,POINT (-0.558055579662323 38.282222747802734),"{'type': 'Point', 'coordinates': [-0.558055579...",2016-04-01,2016-04-01 00:59:59,00:59:59,280,"{'latitude': 38.28, 'longitude': -0.56, 'timez...",...,1014.35,15.13,310,4.03,0.0,0.0,0.0,0,2.01,34
1,LEBL,Place_Barcelona_ElPrat_Airport,BCN,POINT (2.0783333778381348 41.29694366455078),"{'type': 'Point', 'coordinates': [2.0783333778...",2016-04-01,2016-04-01 00:59:59,00:59:59,250,"{'latitude': 41.3, 'longitude': 2.08, 'timezon...",...,1012.15,10.32,39,4.28,0.0,0.0,0.0,0,2.20,48
2,LEBB,Place_Bilbao___Sondica,BIO,POINT (-2.910555601119995 43.301109313964844),"{'type': 'Point', 'coordinates': [-2.910555601...",2016-04-01,2016-04-01 00:59:59,00:59:59,300,"{'latitude': 43.3, 'longitude': -2.91, 'timezo...",...,1022.95,12.71,330,3.58,0.0,0.0,0.0,0,1.79,22
3,LEMD,Place_Madrid_Barajas_Airport,MAD,POINT (-3.560833215713501 40.47222137451172),"{'type': 'Point', 'coordinates': [-3.560833215...",2016-04-01,2016-04-01 00:59:59,00:59:59,180,"{'latitude': 40.47, 'longitude': -3.56, 'timez...",...,1021.25,16.08,351,5.04,0.0,0.0,0.0,0,0.79,38


## Demand Computation

Similar to the previous notebook (5.1), we want to get the traffic demand for each row of our main dataset `dfwx`. This time, we do not focus on airspaces but on airports, so we calculate the demand not by geospatial crossings, but by the departure and destination information contained in the flight plans. 

In the following section, we create a temporary DataFrame that contains for each airport all flights intended to depart or arrive at the airports in focus. Then, for each row in our main DataFrame (`dfwx`), we will go through the temporary DataFrame, count the amount of flights that intend to land or takeoff in the given time slice of the dataset and thereby get its demand.

Technical Comments:
 - _intended_ means that we look only for planned trajectories, not for actual trajectories (DDR M1 data instead of DDR M3 data)

### Demand computation via the RDF store (cancelled)


The actual traffic demand has to be pulled from the data store according to the planned flights. As we have more than 10.000 rows in our dataset, we again have to compute massive geospatial and temporal crossings. And again, we can accomplish this in ways that are "easy" for the RDF store by getting "too much" data with some simple queries and leave the heavy filtering on this side, or we could try to filter on the RDF store side and get an exact result for each row of the dataset. The latter approach would be closer to an production implementation, but due to the experience with the RDF store, the former approach was treid without success.

The following query demonstrates the failed trial to gather the demand data from the triple store. This was cancelled because the 109 triple store only contained a small fraction of flights, and the 107 triple store did not assert the "arrival location" property of a trajectory. 

Now we have two DataFrames with departing flight plans. To calculate total demand for a given airport/time slice in `dfwx`, we iterate through `dfwx`, create a mask on `dfarr` and `dfdep`, respectively, and count the number of flights that fit into that mask.

## Demand Computation via raw data

After it was clear that the demand computation by RDF queries would not be possible, David Knodt (Fraunhofer IAIS) helped my to parse the raw trajectory data, which was analyzed and extracted into two CSV files, icaos.scv and etas.csv.

Those CSV files contain departure airport, destination airport, departure time, and, ind etas.csv, the time over the last node of the trajectory, which represents the ETA (estimated time of arrival) for our puropse.

In [20]:
#load icaos.csv
icaos = pd.read_csv('data/icaos.csv')
icaos['IOBT'] = pd.to_datetime(icaos['IOBT'])
icaos.rename(columns={'DEPARTURE_AERODROME_ICAO_ID': 'dep', 'ARRIVAL_AERODROME_ICAO_ID': 'dest'}, inplace=True)

#load etas.csv
etas = pd.read_csv('data/etas.csv')
etas['IOBT'] = pd.to_datetime(etas['IOBT'])
etas.rename(columns={'MAX(TIMESTAMP)': 'ETA'}, inplace=True)
etas['ETA'] = pd.to_datetime(etas['ETA'])

#merge to a new flightinfo DataFrame
flightinfo = pd.merge(icaos, etas, on=['FLIGHT_UID', 'IOBT'])
flightinfo.head(5)



,FLIGHT_UID,IOBT,dep,dest,ETA
0,150605,2016-04-17 10:20:00,EDDF,EPKK,2016-04-17 11:48:46
1,152348,2016-04-17 11:50:00,EBBR,LIPZ,2016-04-17 13:22:43
2,152351,2016-04-17 17:35:00,ENGM,EKBI,2016-04-17 18:30:40
3,150602,2016-04-17 15:00:00,EBCI,KMDW,2016-04-17 23:20:02
4,148884,2016-04-17 03:20:00,KJFK,EGCC,2016-04-17 10:13:39


In [22]:
dfwx['demand'] = 0.0
dfwx['ratio'] = 0.0
for index, row in dfwx.iterrows():
    aopen = row['sta']
    aclose = row['end']
    airport = row['s']
    
    #boolean masks:
    arrmask = ((flightinfo['ETA'] <= aclose) & (flightinfo['ETA'] >= aopen) & (flightinfo['dest']== airport))
    depmask = ((flightinfo['IOBT'] <= aclose) & (flightinfo['IOBT'] >= aopen) & (flightinfo['dep']==airport))
    
    #apply boolean mas:
    arrivals = flightinfo[arrmask]
    departur = flightinfo[depmask]
    
    # get lentgh of found departure / arrival flight subsets:
    arrcount = len(arrivals)
    depcount = len(departur)
    total = arrcount + depcount
    
    dfwx.at[index, 'demand'] = total

dfwx['ratio'] = dfwx['demand'] / dfwx['cap']

## Inserting the regulation information

The final step will be to mark each row in `dfwx` as 'regulated' or 'not regulated'. We will accomplish this by comparing the Places to the reference locations of the regulations and the times with the active times of the regulations.

### First, we will check what types of regulations are available.


In [23]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?s
WHERE {
  ?s rdfs:subClassOf* :FM_Regulation .
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

dfreg = ts109.query(qry)
dfreg = ts109.clean(dfreg)
f = str(dfreg['s'].tolist())
f


"['FM_Regulation', 'ATC_AccidentCausedRegulation', 'ATC_AerodromeCapacityRegulation', 'ATC_AirportFalicilitiesLimitationRegulation', 'ATC_AirspaceManagementRegulation', 'ATC_Capacity', 'ATC_DeIcingRegulation', 'ATC_EnvironmentalIssueRegulation', 'ATC_Equipment', 'ATC_ImmigrationCustomsHealthRegulation', 'ATC_IndustrialAction', 'ATC_OtherRegulation', 'ATC_OtherRegulationAtDestination', 'ATC_RestrictionAtDepartureRegulation', 'ATC_RestrictionAtDestinationRegulation', 'ATC_RestrictionRegulation', 'ATC_RestrictionStaffShortageRegulation', 'ATC_RestrictionWeatherAtDestinationRegulation', 'ATC_Routing', 'ATC_SecurityRegulation', 'ATC_SpecialEventRegulation', 'ATC_Staffin', 'ATC_WeatherAlternateRegulation', 'ATC_WeatherRegulation', 'NON_ATC_Equipment', 'NON_ATC_IndustrialAction']"

The ebove list shows all possible types of regulations. Not all regulations are relevant for the weather regulation detection scenario. After discussion with CRIDA experts, the following regulations are mainly caused by weather: ATC_WeatherRegulation, ATC_AerodromeCapacityRegulation, ATC_DeIcingRegulation ATC_RestrictionWeatherAtDestinationRegulation. Lets pull these regulations from the data store and prepare a DataFrame with them.

In [24]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?regID ?regType ?trafficVolume ?start ?end ?canceltime ?description ?refLoc ?icao ?coord 
WHERE { 
  ?regID dul:hasRegion          ?trafficVolume ;
         :hasReferenceLocation  ?refLoc ;
         :RegulationDescription ?description ;
         dul:hasTimeInterval    ?time  ;
         :regulationCancelTime  ?ctime ;
         a                      ?regType.

  ?time :TimeStart ?start ;
        :TimeEnd   ?end .

  ?ctime :TimeStart ?canceltime
        
  OPTIONAL {?refLoc :hasICAOcode ?icao.}
  OPTIONAL {?refLoc :hasGeometry/:hasWKT ?coord.}
}
"""

dfreg = ts109.query(qry)
dfreg = ts109.clean(dfreg)
dfreg.head(5)

,regID,regType,trafficVolume,start,end,canceltime,description,refLoc,icao,coord
0,BBDX01N_411,ATC_IndustrialAction,Airspace_LFBBBDX_411,2016-04-01T00:00:00,2016-04-01T04:00:00,2016-03-31T17:44:05,_,Airspace_LFBBBDX_411,NaN,NaN
1,LFLCS01_411,ATC_IndustrialAction,Airspace_LFLCFIS_411,2016-04-01T00:00:00,2016-04-01T05:00:00,2016-03-31T17:03:29,_,Airspace_LFLCFIS_411,NaN,NaN
2,LFMLA01_411,ATC_IndustrialAction,Airspace_LFML_411,2016-04-01T00:00:00,2016-04-01T05:00:00,2016-03-31T18:07:01,_,Place_Marseille_Provence_Airport,LFML,POINT (5.221379177203373 43.44106000164679)
3,LFMLA01_411,ATC_IndustrialAction,Airspace_LFML_411,2016-04-01T00:00:00,2016-04-01T05:00:00,2016-03-31T18:07:01,_,Place_Marseille_Provence_Airport,LFML,POINT (5.215000152587891 43.436668395996094)
4,LFMLA05M_411,ATC_AirspaceManagementRegulation,Airspace_LFML_411,2016-04-05T11:00:00,2016-04-05T12:30:00,2016-04-05T10:21:19,POKER2016_,Place_Marseille_Provence_Airport,LFML,POINT (5.221379177203373 43.44106000164679)


Post-Processing and filtering for only the relevant regulation types:

In [25]:
#convert time values:
dfreg['start'] = pd.to_datetime(dfreg['start']) 
dfreg['end'] = pd.to_datetime(dfreg['end'])
dfreg['canceltime'] = pd.to_datetime(dfreg['canceltime'])

#filter out the relevant regulation types
mask1 = (dfreg['regType'].str.contains('ATC_WeatherRegulation'))
mask2 = (dfreg['regType'].str.contains('ATC_AerodromeCapacityRegulation'))
mask3 = (dfreg['regType'].str.contains('ATC_DeIcingRegulation'))
mask4 = (dfreg['regType'].str.contains('ATC_RestrictionWeatherAtDestinationRegulation'))
dfregf = dfreg[mask1 | mask2 | mask3 |mask4]

#filter out the 'WIP' - WIP means work in progress, e.g. regulations due to construction works)
mask5 = (~ dfregf['description'].str.contains('WIP'))
dfregf = dfregf[mask5]

,regID,regType,trafficVolume,start,end,canceltime,description,refLoc,icao,coord
20,LTFJA02N_411,ATC_AerodromeCapacityRegulation,Airspace_LTFJ_411,2016-04-02 19:20:00,2016-04-02 20:40:00,2016-04-02 12:15:38,_,Place_Istanbul_Sabiha_Gokcen_Airport,NaN,POINT (29.309599142388944 40.90430035539573)
21,LTFJA14E_411,ATC_AerodromeCapacityRegulation,Airspace_LTFJ_411,2016-04-14 02:00:00,2016-04-14 06:00:00,2016-04-13 23:49:48,_,Place_Istanbul_Sabiha_Gokcen_Airport,NaN,POINT (29.309599142388944 40.90430035539573)
22,LTFJA27M_411,ATC_AerodromeCapacityRegulation,Airspace_LTFJ_411,2016-04-27 07:00:00,2016-04-27 09:00:00,2016-04-27 01:51:36,_,Place_Istanbul_Sabiha_Gokcen_Airport,NaN,POINT (29.309599142388944 40.90430035539573)
26,EDDFA01_411,ATC_WeatherRegulation,Airspace_EDDF_411,2016-04-01 04:40:00,2016-04-01 07:20:00,2016-04-01 05:52:57,STRONG WINDS_,Place_Frankfurt_Main_Airport,NaN,POINT (8.571822869076076 50.05067708952074)
28,EDDFA12M_411,ATC_WeatherRegulation,Airspace_EDDF_411,2016-04-12 04:40:00,2016-04-12 08:40:00,2016-04-12 04:03:22,EXPECTED TS_,Place_Frankfurt_Main_Airport,NaN,POINT (8.571822869076076 50.05067708952074)
30,EDDFA24N_411,ATC_WeatherRegulation,Airspace_EDDF_411,2016-04-24 16:40:00,2016-04-24 20:00:00,2016-04-24 17:55:59,CB THUNDERSTORM_,Place_Frankfurt_Main_Airport,NaN,POINT (8.571822869076076 50.05067708952074)
31,EDDFA26A_411,ATC_WeatherRegulation,Airspace_EDDF_411,2016-04-26 16:20:00,2016-04-26 19:00:00,2016-04-26 16:46:00,TS AND CB'S_,Place_Frankfurt_Main_Airport,NaN,POINT (8.571822869076076 50.05067708952074)
32,EDDFA27A_411,ATC_WeatherRegulation,Airspace_EDDF_411,2016-04-27 16:45:00,2016-04-27 19:20:00,2016-04-27 17:55:49,CB_,Place_Frankfurt_Main_Airport,NaN,POINT (8.571822869076076 50.05067708952074)
39,LTBAA02A_411,ATC_AerodromeCapacityRegulation,Airspace_LTBA_411,2016-04-02 15:40:00,2016-04-02 17:00:00,2016-04-02 11:01:30,_,Place_Istanbul_Ataturk_Airport,LTBA,POINT (28.8195493087893 40.977838817779705)
40,LTBAA02A_411,ATC_AerodromeCapacityRegulation,Airspace_LTBA_411,2016-04-02 15:40:00,2016-04-02 17:00:00,2016-04-02 11:01:30,_,Place_Istanbul_Ataturk_Airport,LTBA,POINT (28.814167022705078 40.976112365722656)


In [26]:
dfregf.describe()

,regID,regType,trafficVolume,start,end,canceltime,description,refLoc,icao,coord
count,175,175,175,175,175,175,175,175,112,149
unique,118,2,58,107,104,118,46,51,20,51
top,EDDHA10A_411,ATC_WeatherRegulation,Airspace_EGSS_411,2016-04-29 16:00:00,2016-04-12 09:00:00,2016-04-10 16:22:17,_,Place_Madrid_Barajas_Airport,ENGM,POINT (-0.4531566520633094 51.47099587999384)
freq,3,115,10,5,6,3,46,12,12,11
first,NaN,NaN,NaN,2016-04-01 04:40:00,2016-04-01 07:00:00,2016-03-31 15:55:07,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2016-04-30 15:40:00,2016-04-30 17:00:00,2016-04-30 11:10:09,NaN,NaN,NaN,NaN


Now we will append the regulation information to the main dataset `dfwx`. We will loop through `dfwx` and check for each row if a regulation at that reference location exists.

In [27]:
dfwx['regulated'] = 0

for index, wxrow in dfwx.iterrows():
    for index2, regrow in dfregf.iterrows():
        if wxrow['s'] == regrow['refLoc']:
            #check if reg begins within opening period:
            regula = 0
            if ((regrow['start'].time() >= wxrow['sta'].time()) & (regrow['start'].time() <= wxrow['end'].time()) ):
                regula = 1
            #check if reg ends within opening period:
            if ((regrow['end'].time() >= wxrow['sta'].time()) & (regrow['end'].time() <= wxrow['end'].time())  ):
                regula = 1
            #check if reg period covers opening period:
            if ((regrow['start'].time() <= wxrow['sta'].time()) & (regrow['end'].time() >= wxrow['end'].time())  ):
                regula = 1
            #update data:
            if regula == 1:
                dfwx.at[index, 'regulated'] = 1

In [28]:
dfwx.describe()

,duration,rwys,humidity,dewPoint,temperature,pressure,visibility,windBearing,windSpeed,windGust,precipIntensity,precipProbability,storm,crosswind,cap,demand,ratio,regulated
count,100,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.0,100.00,100.00,100.0,100.00,100.00,100.0,100.0,100.0
mean,0 days 00:59:59,267.10,0.60,2.51,11.16,1016.14,11.69,220.29,3.92,0.0,0.03,0.01,0.0,1.65,47.42,0.0,0.0,0.1
std,0 days 00:00:00,44.14,0.22,4.67,4.06,3.49,3.01,118.43,2.09,0.0,0.21,0.05,0.0,1.66,22.05,0.0,0.0,0.3
min,0 days 00:59:59,180.00,0.20,-3.84,1.71,1011.71,5.41,0.00,0.03,0.0,0.00,0.00,0.0,0.00,22.00,0.0,0.0,0.0
25%,0 days 00:59:59,240.00,0.40,-2.13,8.89,1013.75,9.99,106.25,2.31,0.0,0.00,0.00,0.0,0.27,34.00,0.0,0.0,0.0
50%,0 days 00:59:59,280.00,0.59,2.26,11.83,1014.57,10.54,280.50,4.02,0.0,0.00,0.00,0.0,1.15,37.00,0.0,0.0,0.0
75%,0 days 00:59:59,300.00,0.81,7.40,12.78,1018.31,15.05,310.25,5.48,0.0,0.00,0.00,0.0,2.65,66.00,0.0,0.0,0.0
max,0 days 00:59:59,320.00,0.94,10.01,20.63,1022.96,16.09,359.00,8.50,0.0,1.92,0.38,0.0,7.46,100.00,0.0,0.0,1.0




TODO

The mean of the column "regulated" gives us a hint how unbalanced the problem set is. This has to be considered in the next notebook!

## Saving the results

Now we have a new column, "regulated", which represents if a regulation took place at the time slice of a given row in the dataset. This is the final result of our computations and will be used in the ML phase. Now, the dataset is complete and we can save it to the system and continue with the next workbook, 5.4 Learning from the Data.

In [29]:
dfwx.to_csv('data/dffinal.csv')